In [24]:
from copy import deepcopy
import dill
import logging
import os
from pprint import pprint

from human_aware_rl.rllib.rllib import gen_trainer_from_params, load_agent, OvercookedMultiAgent
from human_aware_rl.ppo.ppo_rllib_client import my_config
from overcooked_ai_py.agents.benchmarking import AgentEvaluator
from overcooked_ai_py.mdp.actions import Action
from overcooked_ai_py.mdp.layout_generator import DEFAILT_PARAMS_SCHEDULE_FN, LayoutGenerator, MDPParamsGenerator

RAY_DIRECTORY = os.path.expanduser("~/ray_results")

In [25]:
def checkpoint_path(run_name, seed=0, checkpoint_num=1):
    run_dir = sorted([r for r in os.listdir(RAY_DIRECTORY) if f"{run_name}_{seed}" in r])[0]
    cp_path = f"{RAY_DIRECTORY}/{run_dir}/checkpoint_{checkpoint_num}/checkpoint-{checkpoint_num}"
    return cp_path

def load_params(run_name, seed=0):
    cp_path = checkpoint_path(run_name, seed)
    params_path = "/".join(cp_path.split("/")[:-1]) + "/config.pkl"
    params = dill.load(open(params_path, "rb"))
    return params

def load_env(run_name, seed=0):
    params = load_params(run_name, seed)
    return OvercookedMultiAgent.from_config(params["environment_params"])

def load_agents(run_name, seeds, checkpoint_num):        
    agents = {}
    for seed in seeds:
        agents[seed] = load_agent(
            checkpoint_path(run_name, seed=seed, checkpoint_num=checkpoint_num), 
            policy_id="ppo", 
            agent_index=-1  # set to 0 or 1 when initializing episode
        )
    return agents

In [26]:
def self_play(mdp, agent, num_games=100, rnd_obj_prob_thresh=0.0):
    params = {
        'horizon': 400, 
        'mlam_params': {
            'start_orientations': False,
            'wait_allowed': False,
            'counter_goals': [],
            'counter_drop': [],
            'counter_pickup': [],
            'same_motion_goals': True
        }
    }
    # start_state_fn = mdp.get_random_start_state_fn(random_start_pos=False, rnd_obj_prob_thresh=rnd_obj_prob_thresh)
    
    # TODO change to AgentEvaluator.from_mdp_lst
    trajs = AgentEvaluator.from_mdp(mdp, params).get_agent_pair_trajs(
        a0=agent, num_games=num_games
    )
    print(f"agent self-play: {trajs[0]['ep_returns'].mean()}")
    
    return trajs

def cross_play(mdp, agent_0, agent_1, num_games=100, rnd_obj_prob_thresh=0.0):
    params = {
        'horizon': 400, 
        'mlam_params': {
            'start_orientations': False,
            'wait_allowed': False,
            'counter_goals': [],
            'counter_drop': [],
            'counter_pickup': [],
            'same_motion_goals': True
        }
    }
    # start_state_fn = mdp.get_random_start_state_fn(random_start_pos=False, rnd_obj_prob_thresh=rnd_obj_prob_thresh)
    
    # TODO change to AgentEvaluator.from_mdp_lst
    trajs_0_0 = AgentEvaluator.from_mdp(mdp, params).get_agent_pair_trajs(
        a0=agent_0, num_games=num_games
    )
    print(f"agent 0 self-play: {trajs_0_0[0]['ep_returns'].mean()}")

    trajs_1_1 = AgentEvaluator.from_mdp(mdp, params).get_agent_pair_trajs(
        a0=agent_1, num_games=num_games
    )
    print(f"agent 1 self-play: {trajs_1_1[0]['ep_returns'].mean()}")

    trajs_0_1 = AgentEvaluator.from_mdp(mdp, params).get_agent_pair_trajs(
        a0=agent_0, a1=agent_1, num_games=num_games
    )
    print(f"cross-play: {trajs_0_1[0]['ep_returns'].mean()}")
    
    return trajs_0_0, trajs_1_1, trajs_0_1

In [8]:
# mdp = LayoutGenerator(MDPParamsGenerator(DEFAILT_PARAMS_SCHEDULE_FN)).generate_padded_mdp()

run_name = "room"
seed = 1

mdp = load_env(run_name, seed).base_env.mdp
print(load_params(run_name, seed))

for checkpoint_num in [501, 1001, 1501, 2001, 2501, 3001, 3501]:
    path_0 = checkpoint_path(run_name, seed=seed, checkpoint_num=checkpoint_num)
    # path_1 = checkpoint_path(run_name, seed=seed, checkpoint_num=checkpoint_num)
    num_games = 100
    rnd_obj_prob_thresh = 0.0

    agent_0 = load_agent(path_0, policy_id="ppo", agent_index=0)
    # agent_1 = load_agent(path_1, policy_id="ppo", agent_index=1)
    trajs = self_play(mdp, agent_0, num_games, rnd_obj_prob_thresh)
    #trajs_0_0, trajs_1_1, trajs_0_1 = cross_play(mdp, agent_0, agent_1, num_games, rnd_obj_prob_thresh)

{'model_params': {'use_lstm': False, 'NUM_HIDDEN_LAYERS': 3, 'SIZE_HIDDEN_LAYERS': 64, 'NUM_FILTERS': 25, 'NUM_CONV_LAYERS': 3, 'CELL_SIZE': 256, 'D2RL': False}, 'training_params': {'num_workers': 2, 'train_batch_size': 12000, 'sgd_minibatch_size': 2000, 'rollout_fragment_length': 400, 'num_sgd_iter': 8, 'lr': 0.001, 'lr_schedule': None, 'grad_clip': 0.1, 'gamma': 0.99, 'lambda': 0.98, 'vf_share_layers': True, 'vf_loss_coeff': 0.0001, 'kl_coeff': 0.2, 'clip_param': 0.05, 'num_gpus': 0, 'seed': 1, 'evaluation_interval': 50, 'entropy_coeff_schedule': [[0, 0.2], [300000.0, 0.1]], 'eager': False, 'log_level': 'WARN'}, 'environment_params': {'env_params': {'horizon': 400, 'mlam_params': {'start_orientations': False, 'wait_allowed': False, 'counter_goals': [], 'counter_drop': [], 'counter_pickup': [], 'same_motion_goals': True}}, 'multi_agent_params': {'reward_shaping_factor': 1.0, 'reward_shaping_horizon': 2500000.0, 'use_phi': True, 'bc_schedule': [[0, 0], [inf, 0]]}, 'mdp_params': {'layou

/home/anchorwatt/miniconda3/envs/harl/lib/python3.7/site-packages/tensorflow/python/keras/legacy_tf_layers/core.py:329: UserWarning: `tf.layers.flatten` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Flatten` instead.
  warnings.warn('`tf.layers.flatten` is deprecated and '
/home/anchorwatt/miniconda3/envs/harl/lib/python3.7/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1692: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
/home/anchorwatt/miniconda3/envs/harl/lib/python3.7/site-packages/tensorflow/python/keras/legacy_tf_layers/core.py:329: UserWarning: `tf.layers.flatten` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Flatten` instead.
  warnings.warn('`tf.layers.flatten` is deprecated and '
/home/anchorwatt/miniconda3/envs/harl/lib/python3.7/site-packages/tensorflow/pyth

Skipping trajectory consistency checking because MDP was recognized as variable. Trajectory consistency checking is not yet supported for variable MDPs.
agent self-play: 66.2





 1001


/home/anchorwatt/miniconda3/envs/harl/lib/python3.7/site-packages/tensorflow/python/keras/legacy_tf_layers/core.py:329: UserWarning: `tf.layers.flatten` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Flatten` instead.
  warnings.warn('`tf.layers.flatten` is deprecated and '
/home/anchorwatt/miniconda3/envs/harl/lib/python3.7/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1692: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
/home/anchorwatt/miniconda3/envs/harl/lib/python3.7/site-packages/tensorflow/python/keras/legacy_tf_layers/core.py:329: UserWarning: `tf.layers.flatten` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Flatten` instead.
  warnings.warn('`tf.layers.flatten` is deprecated and '
/home/anchorwatt/miniconda3/envs/harl/lib/python3.7/site-packages/tensorflow/pyth

Skipping trajectory consistency checking because MDP was recognized as variable. Trajectory consistency checking is not yet supported for variable MDPs.
agent self-play: 71.4





 1501


...

Skipping trajectory consistency checking because MDP was recognized as variable. Trajectory consistency checking is not yet supported for variable MDPs.
agent self-play: 75.6





 3001


/home/anchorwatt/miniconda3/envs/harl/lib/python3.7/site-packages/tensorflow/python/keras/legacy_tf_layers/core.py:329: UserWarning: `tf.layers.flatten` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Flatten` instead.
  warnings.warn('`tf.layers.flatten` is deprecated and '
/home/anchorwatt/miniconda3/envs/harl/lib/python3.7/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1692: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
/home/anchorwatt/miniconda3/envs/harl/lib/python3.7/site-packages/tensorflow/python/keras/legacy_tf_layers/core.py:329: UserWarning: `tf.layers.flatten` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Flatten` instead.
  warnings.warn('`tf.layers.flatten` is deprecated and '
/home/anchorwatt/miniconda3/envs/harl/lib/python3.7/site-packages/tensorflow/pyth

Skipping trajectory consistency checking because MDP was recognized as variable. Trajectory consistency checking is not yet supported for variable MDPs.
agent self-play: 84.2





 3501


/home/anchorwatt/miniconda3/envs/harl/lib/python3.7/site-packages/tensorflow/python/keras/legacy_tf_layers/core.py:329: UserWarning: `tf.layers.flatten` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Flatten` instead.
  warnings.warn('`tf.layers.flatten` is deprecated and '
/home/anchorwatt/miniconda3/envs/harl/lib/python3.7/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1692: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
/home/anchorwatt/miniconda3/envs/harl/lib/python3.7/site-packages/tensorflow/python/keras/legacy_tf_layers/core.py:329: UserWarning: `tf.layers.flatten` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Flatten` instead.
  warnings.warn('`tf.layers.flatten` is deprecated and '
/home/anchorwatt/miniconda3/envs/harl/lib/python3.7/site-packages/tensorflow/pyth

Skipping trajectory consistency checking because MDP was recognized as variable. Trajectory consistency checking is not yet supported for variable MDPs.
agent self-play: 80.6


In [23]:
from overcooked_ai_py.visualization.state_visualizer import *

StateVisualizer().display_rendered_trajectory(
    trajs[0], img_directory_path="/home/anchorwatt/traj_0_0"
)

pygame 1.9.5
Hello from the pygame community. https://www.pygame.org/contribute.html


interactive(children=(IntSlider(value=0, description='timestep', max=399), Output()), _dom_classes=('widget-in…